# Molecular network lifetimes with kugupu

This notebook shows how the `kugupu.time` submodule can be used to investigate the time dependence of molecular networks.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import kugupu as kgp
import numpy as np

In [ ]:
res = kgp.load_results('./datafiles/results.hdf5')

In [ ]:
res.frames

The `kugupu.time.determine_lifetimes` function calculates the duration of each coupling between two fragments.
The function returns a one dimensional array of the measured lifetimes of the coupling,
as well as a boolean array of if the end of the lifetime was observed.
The latter is important in avoiding right censorship when determining the lifetime.

In [ ]:
durations, observations = kgp.time.determine_lifetimes(res.H_frag, 0.001)

We can plot a histogram of the lifetimes to get an idea of how long our molecules are staying coupled.
Here we see that most are only briefly coupled.

In [ ]:
fig, ax = plt.subplots()

ax.hist(durations)

ax.set_title('Distribution of observed coupling lifetimes')
ax.set_xlabel('Duration')
ax.set_ylabel('Count')

We can do a more detailed analysis by fitting a Kaplan-Meier estimator to the data.
This is done using an implementation from the `lifelines` Python package.

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import rmst_plot

kmf = KaplanMeierFitter()
kmf_fit = kmf.fit(durations, event_observed=observations)  

This can be plotted to show the estimated rate of decay of a given coupling over time.
Here we can see that after 5 frames, only 20% of couplings are expected to still be "alive".

In [ ]:
fig, ax = plt.subplots()
rmst_plot(kmf_fit,ax=ax)


ax.set_ylabel('Fraction alive')
ax.set_xlabel('Time (frames)')